In [30]:
#!pip install pymysql #安装数据库所需的包

In [ ]:
#!pip install rqdatac #安装米筐所需的包

In [2]:
import pymysql
import pandas as pd

In [3]:
from rqdatac import *
import rqdatac as rq

# rq.init('手机号', '密码')
rq.init('13132503161', '138ricezhao')

### 一、使用上证50指数2020-01-04至2020-05-04的数据作为例子，检验两个数据是否一致

In [4]:
sql=''' select * from  cn_stock_index.daily_quote where index_code="000016"
        and trade_date between "2020-01-04 00:00:00" and "2020-05-04 23:59:59" '''

创建提取数据库数据的函数，每次写好sql之后可直接提取。
由于间隔过长时与数据库的连接会断开，在每次获取数据之后关闭连接。

In [5]:
def retrieve_data(sql): # 创建连接数据库提取数据的函数
    conn = pymysql.connect(
                host = '172.19.3.249',
                port = 3306,
                user ='ktruc002' ,
                passwd = '35442fed',
                db = 'cn_bond',
                charset = 'gbk'
            )
    data_sql=pd.read_sql(sql, con= conn)
    print('success')
    conn.close()
    return data_sql

In [6]:
data1=retrieve_data(sql)
data1.dropna(axis=1,how='any',inplace=True)

success


首先检视数据shape，判断是否一致

In [9]:
data1.shape

(77, 17)

In [10]:
data1rq=rq.get_price('000016.XSHG', start_date='2020-01-04', end_date='2020-05-04', frequency='1d')

In [11]:
data1rq.shape

(77, 7)

两组数据均为77条记录，则直接处理数据并判断是否相同

In [12]:
data1=data1[['open','last']]   #选取需要的数据列
data1rq=data1rq[['open','close']]

In [13]:
def cleanup(df,columns): #定义函数处理dataframe，使行与列标签相同，方便下一步比较
    df=df.reset_index(drop=True)
    df.columns=columns
    return df

In [14]:
data1rq.head()

,open,close
date,,
2020-01-06,3062.2801,3056.8359
2020-01-07,3063.7464,3074.0152
2020-01-08,3058.3585,3037.8525
2020-01-09,3060.7743,3067.5456
2020-01-10,3079.0765,3067.8810


In [15]:
data1rq=cleanup(data1rq,data1.columns)

In [16]:
data1rq.head()

,open,last
0,3062.2801,3056.8359
1,3063.7464,3074.0152
2,3058.3585,3037.8525
3,3060.7743,3067.5456
4,3079.0765,3067.8810


使用assert_frame_equal来判断数据是否相同

In [17]:
from pandas.testing import assert_frame_equal

In [18]:
assert_frame_equal(data1, data1rq,check_names=False) 
# 比较数据是否有不同的值，大致比较，会忽略微小值差异。可通过check_exact=True或check_less_precise= bool or int来修改比较的精细程度
# 若无错误提示则两组数据相同

无错误提示，两组数据相同。

### 二、使用中国宝安股票2020-01-04至2020-05-04的数据作为例子，检验两个数据是否一致

In [19]:
sql2='''select * from  cn_stock_quote.daily_quote where code="000009" and type="A股"
        and trade_date between "2020-01-04 00:00:00" and "2020-05-04 23:59:59" '''

In [20]:
data2=retrieve_data(sql2)
data2.dropna(axis=1,how='any',inplace=True)

success


In [21]:
data2.shape

(75, 28)

In [22]:
data2rq=rq.get_price('000009.XSHE', start_date='2020-01-04', end_date='2020-05-04', frequency='1d')
data2rq.reset_index(inplace=True)

In [23]:
data2rq.shape

(77, 10)

shape不一致，则首先查找前者缺失的数据

构建一个函数比较两组数据，直接输出缺失的行。 由于使用了pd.merge，参数为left，需将条数多的数据至于左边。

In [25]:
def anti_join(x, y, left, right):  
# 查找其中一组数据的缺失值,行数大的置于左
    dfmerge = pd.merge(x, y, how='left', left_on=left, right_on=right)   
    diff =  dfmerge[dfmerge.isnull().T.any()]  
    return diff

In [26]:
df_lost=anti_join(data2rq,data2,'date','trade_date')
df_lost

,date,volume_x,high_x,close,num_trades,open_x,limit_up,low_x,limit_down,total_turnover,...,amount_original_currency,currency,amount_rmb,turnover_rate,issue_share,float_share,input_time,modify_time,valid,update_stamp
72,2020-04-24,45956445.0,6.41,6.17,21644.0,6.28,6.92,6.14,5.66,287834692.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN
75,2020-04-29,67959741.0,6.44,6.28,31427.0,6.40,7.18,6.21,5.88,429545554.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN


删除这两行之后，选取需要比较的数据来进行比较，判断剩余的数据是否一致：

In [27]:
data2=data2[['open','last']]
data2rq=data2rq[['open','close']].drop(df_lost.index)

In [28]:
data2rq=cleanup(data2rq,data2.columns)

In [29]:
assert_frame_equal(data2, data2rq,check_names=False)

无报错，则证明两组数据除了RiceQuant多出了两行数据之外，其他均一致。

# 三、总结

以上展示了检验两组数据是否一致的大致过程，并使用几个简单的函数来简化。虽然仅使用了两个例子，但不同的数据，检验过程基本一致。且在第二个例子中发现了数据条数不一致的情况。经百度发现4-24与4-29均是交易日，且中国宝安均有交易数据，并未停牌。故初步推测是数据库数据缺失。